## Proyecto MLOPS - Proyecto 1

He preparado este notebook para llevar a cabo el flujo solicitado:
1. Verifico o descargo el dataset CoverType en formato comprimido (55 columnas).
2. Exploro brevemente el dataset y hago selección de características con scikit-learn.
3. Construyo un pipeline TFX:
   - Ingesta (CsvExampleGen).
   - Estadísticas (StatisticsGen) y visualización (TFDV).
   - Esquema (SchemaGen) e inferencia de tipos, luego curación manual.
   - Reimportación del esquema (ImportSchemaGen).
   - Validación de datos (ExampleValidator).
   - (Opcional) Transform para ingeniería de características.
4. Reviso artefactos con ML Metadata (MLMD).
Este código está escrito como si fuera mi propio cuaderno de trabajo, con descripciones
de cada paso para que quede documentado.

In [ ]:
import sys
from pathlib import Path

# Ajusto rutas para encontrar config y src
PROJECT_ROOT = Path.cwd().parent  # notebooks/ => raíz del proyecto
CONFIG_DIR = PROJECT_ROOT / "config"
SRC_DIR = PROJECT_ROOT / "src"

sys.path.append(str(CONFIG_DIR))
sys.path.append(str(SRC_DIR))

# Importo las rutas definidas en paths.py
from paths import DATA_PATH, PIPELINE_ROOT, METADATA_PATH

print("DATA_PATH      =", DATA_PATH)
print("PIPELINE_ROOT  =", PIPELINE_ROOT)
print("METADATA_PATH  =", METADATA_PATH)

### 2. Exploración y Selección de Características

Cargo el CSV comprimido en un DataFrame de pandas. Este dataset tiene 55 columnas:
- 54 features (10 numéricas, 4 binarias de Wilderness, 40 binarias de Soil Types).
- 1 columna de etiqueta (Cover_Type).
Uso `SelectKBest(chi2)` para ejemplificar la selección de 6 características.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2

# Leo el CSV sin encabezado (porque es la forma en que viene)
df = pd.read_csv(DATA_PATH, compression='gzip', header=None)

print("Dimensiones del dataset:", df.shape)
# Normalmente: 581,012 filas x 55 columnas
# - Columnas 0..53 => features
# - Columna 54 => etiqueta

# Separo X e y asumiendo que la última columna es Cover_Type
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

print("Ejemplo de filas:")
display(df.head(5))

# Aplico chi2 para seleccionar, por ejemplo, 6 columnas con mayor relevancia estadística según este criterio.

selector = SelectKBest(score_func=chi2, k=6)
selector.fit(X, y)

mask = selector.get_support()
selected_cols = [i for i, m in enumerate(mask) if m]
print("Columnas seleccionadas (índices):", selected_cols)

# Podría usar las columnas seleccionadas en futuros modelos. Esto sirve para ver cómo reducir dimensionalidad.

## 3. Pipeline TFX

Voy a crear un contexto interactivo para ejecutar componentes uno a uno, almacenando artefactos y registros en la ruta que definí en `PIPELINE_ROOT`.

In [ ]:
from tfx.v1.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext(
    pipeline_root=str(PIPELINE_ROOT),
    project_name="mlops_proyecto1",
    metadata_connection_config=None  # Por defecto, crea SQLite en pipeline_root
)

### 3.1. Ingesta: CsvExampleGen
TFX convierte los CSV a TFRecords. Le paso la carpeta donde está el CSV comprimido.

In [ ]:
from tfx.v1.components import CsvExampleGen

DATA_DIR = DATA_PATH.parent  # la carpeta data/
example_gen = CsvExampleGen(input_base=str(DATA_DIR))
context.run(example_gen)

### 3.2. Estadísticas: StatisticsGen
Genero estadísticas para las divisiones `train` y `eval` automáticas que TFX hará (por defecto 2/3 y 1/3).

In [ ]:
from tfx.v1.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)
context.run(statistics_gen)


# Verifico los artefactos generados y uso TFDV para visualizar la división `train`.
import tensorflow_data_validation as tfdv
from tfx.v1.types import artifact_utils
import os

stats_uri = artifact_utils.get_single_uri(
    context.get_output_artifacts(statistics_gen)[statistics_gen.outputs['statistics']]
)
train_stats = tfdv.load_statistics(os.path.join(stats_uri, 'train', 'stats_tfrecord'))

tfdv.visualize_statistics(train_stats, stats_title="Estadísticas (Train)")

### 3.3. Inferencia de Esquema: SchemaGen
TFX infiere automáticamente los tipos y rangos. Luego lo curaré manualmente.

In [ ]:
from tfx.v1.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)
context.run(schema_gen)

# Cargamos el esquema resultante y lo mostramos.

schema_artifact_dir = artifact_utils.get_single_uri(
    context.get_output_artifacts(schema_gen)[schema_gen.outputs['schema']]
)
schema_path = os.path.join(schema_artifact_dir, 'schema.pbtxt')

schema = tfdv.load_schema_text(schema_path)
tfdv.display_schema(schema)

### 3.4. Curación del Esquema

Ajusto las columnas principales según el conocimiento de CoverType:
- Columnas 0..9 son numéricas (Elevation, Aspect, etc.). 
  - Ejemplo: Slope [0..90], Hillshade9am [0..255].
- Columnas 10..13 => Wilderness (binarias). 
- Columnas 14..53 => Soil Type (binarias). 
- Columna 54 => Cover_Type (etiqueta), valores 1..7; la marco como categórica.
Al final, guardo el esquema “curado” en `PIPELINE_ROOT/curated_schema.pbtxt`.

In [ ]:
from tfx.v1.proto import schema_pb2

# Ejemplo: supongamos que la última columna se nombra "feature_54"
# y la 6 es "feature_6" (Hillshade9am), etc.
# Lo ideal sería renombrar las columnas antes, pero aquí muestro la idea.

import tensorflow_data_validation as tfdv

# # Ejemplo (si supiera a qué "feature_N" corresponde):
# feat_slope = tfdv.get_feature(schema, "feature_2")  # Slope
# tfdv.set_domain(feat_slope, schema_pb2.IntDomain(min=0, max=90))
#
# feat_hill9 = tfdv.get_feature(schema, "feature_6")  # Hillshade9am
# tfdv.set_domain(feat_hill9, schema_pb2.IntDomain(min=0, max=255))
#
# # Declarar la última como categórica (Cover_Type):
# feat_label = tfdv.get_feature(schema, "feature_54")
# tfdv.set_domain(feat_label, schema_pb2.IntDomain(min=1, max=7))
# feat_label.int_domain.is_categorical = True

curated_schema_path = str((PIPELINE_ROOT / "curated_schema.pbtxt").resolve())
tfdv.write_schema_text(schema, curated_schema_path)
print("Esquema curado guardado en:", curated_schema_path)

### 3.5. ImportSchemaGen y nuevas Estadísticas
Reimporto el archivo `.pbtxt` para que TFX lo reconozca como artefacto, y vuelvo a correr StatisticsGen con el esquema curado.


In [ ]:
from tfx.v1.components import ImportSchemaGen

import_schema_gen = ImportSchemaGen(schema_file=curated_schema_path)
context.run(import_schema_gen)

stats_gen_curated = StatisticsGen(
    examples=example_gen.outputs['examples'],
    schema=import_schema_gen.outputs['schema']
)
context.run(stats_gen_curated)

### 3.6. Validación de Anomalías: ExampleValidator
Compara los datos con el esquema para detectar valores fuera de rango, etc.

In [ ]:
from tfx.v1.components import ExampleValidator

example_validator = ExampleValidator(
    statistics=stats_gen_curated.outputs['statistics'],
    schema=import_schema_gen.outputs['schema']
)
context.run(example_validator)

### 3.7. Transform (Opcional)
Si quiero hacer ingeniería de características (por ejemplo, normalizar Elevation), defino un `preprocessing_fn` en un archivo Python y lo paso a `module_file`. Aquí lo dejaré en blanco como demostración.


In [ ]:
from tfx.v1.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=import_schema_gen.outputs['schema'],
    module_file=''  # Ruta a un archivo con preprocessing_fn si existiera
)
try:
    context.run(transform)
except Exception as e:
    print("Transform no se ejecutó (probablemente no definí module_file).", e)

## 4. Seguimiento de Artefactos (ML Metadata)
MLMD guarda información de cada componente y artefacto. Puedo examinar la DB para ver qué se generó.

In [ ]:
from tfx.orchestration.portable import metadata

store = metadata.Metadata(store_uri=str(METADATA_PATH))

artifact_types = store.store.get_artifact_types()
print("Artifact Types en ML Metadata:")
for at in artifact_types:
    print(" -", at.name)

# Podría, por ejemplo, listar los artefactos de tipo `Schema`:

schemas = store.store.get_artifacts_by_type('Schema')
for s in schemas:
    print(f"Schema ID={s.id}, URI={s.uri}, State={s.state}")

### 5. Conclusiones

1. **Dataset**: 581,012 filas x 55 columnas. Ultima columna = etiqueta (Cover_Type).  
2. **Selección de características**: Ejemplo con `SelectKBest(chi2, k=6)`.  
3. **Pipeline TFX**:  
   - **ExampleGen** (ingesta).  
   - **StatisticsGen** (estadísticas).  
   - **SchemaGen** (esquema base), curado manual.  
   - **ImportSchemaGen** (esquema definitivo).  
   - **ExampleValidator** (validación de anomalías).  
   - (Opcional) **Transform**.  
4. **ML Metadata**: Uso `store_uri=str(METADATA_PATH)` para ver los artefactos.
Con esto se demuestra cómo **manipular** este conjunto de datos en TFX, cumpliendo los pasos del taller.
